In [260]:
# import following packages first
%matplotlib inline
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [261]:
from os import listdir
from os.path import join
source_path = '/Users/meghnadiwan/Downloads/Hackathon/COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/'

In [262]:
files = [f for f in listdir(source_path) if f.endswith('.csv')]

In [263]:
df = pd.concat([pd.read_csv(join(source_path, f_name)) for f_name in files], sort = False)

#subset to US only
df = df[df['Country/Region'] == "US"]

In [264]:
# keep on columns needed
cols = ['Province/State', 'Country/Region', 'Last Update', 'Confirmed',
       'Deaths', 'Recovered','Latitude', 'Longitude']
df = df[cols]

In [265]:
# convert date to datetime
df['Last Update'] = pd.to_datetime(df['Last Update'])

In [266]:
# remove (From Diamond Princess)
df['Province/State'] = df['Province/State'].str.replace("(From Diamond Princess)", "")
df['Province/State'] = df['Province/State'].str.strip(" ()")

In [267]:
#Split Province/State into Province and State
split = df['Province/State'].str.split(',', expand=True)
df['Province'] = split[0].str.strip()
df['State'] = split[1].str.strip()

In [268]:
# Replace missing State values using dictionary below
st_dict = {'Unassigned Location': 'Unassigned Location', 
              'Washington':'WA', 'New York':'NY', 'California':'CA',
              'Massachusetts':'MA','Georgia':'GA','Colorado':'CO', 
              'Florida':'FL', 'New Jersey':'NJ', 'Oregon':'OR', 
              'Texas':'TX', 'Illinois':'IL', 'Pennsylvania':'PA', 
              'Iowa':'IA', 'Maryland':'MD', 'North Carolina':'NC',
              'South Carolina':'SC', 'Tennessee':'TN', 'Virginia':'VA', 
              'Arizona':'AZ', 'Indiana':'IN','Kentucky':'KY', 
              'District of Columbia':'D.C.', 'Nevada':'NV', 'New Hampshire':'NH',
              'Minnesota':'MN', 'Nebraska':'NE', 'Ohio':'OH', 'Rhode Island':'RI', 
              'Wisconsin':'WI','Connecticut':'CT', 'Hawaii':'HI', 'Oklahoma':'OK', 
              'Utah':'UT', 'Kansas':'KS', 'Louisiana':'LA','Missouri':'MO', 
              'Vermont':'VT', 'Alaska':'AK', 'Arkansas':'AR', 'Delaware':'DE', 
              'Idaho':'ID','Maine':'ME', 'Michigan':'MI', 'Mississippi':'MS', 
              'Montana':'MT', 'New Mexico':'NM','North Dakota':'ND', 'South Dakota':'SD', 
              'West Virginia':'WV', 'Wyoming':'WY','Alabama':'AL', 'Puerto Rico':'PR', 
              'Guam': 'GU','Virgin Islands':'VI', 'United States Virgin Islands': 'VI', 'Chicago':'IL'}
df['State'] = df['State'].fillna(df.Province.map(st_dict))

In [269]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1560 entries, 36 to 254
Data columns (total 10 columns):
Province/State    1560 non-null object
Country/Region    1560 non-null object
Last Update       1560 non-null datetime64[ns]
Confirmed         1560 non-null float64
Deaths            1530 non-null float64
Recovered         1530 non-null float64
Latitude          1186 non-null float64
Longitude         1186 non-null float64
Province          1560 non-null object
State             1529 non-null object
dtypes: datetime64[ns](1), float64(5), object(4)
memory usage: 134.1+ KB


In [270]:
df.State.unique()

array(['Unassigned Location', 'IL', 'CA', 'MA', 'WI', 'TX', 'WA', 'AZ',
       'NE', 'NY', nan, 'GA', 'CO', 'FL', 'NJ', 'OR', 'PA', 'IA', 'MD',
       'NC', 'SC', 'TN', 'VA', 'IN', 'KY', 'D.C.', 'NV', 'NH', 'MN', 'OH',
       'RI', 'CT', 'HI', 'OK', 'UT', 'KS', 'LA', 'MO', 'VT', 'AK', 'AR',
       'DE', 'ID', 'ME', 'MI', 'MS', 'MT', 'NM', 'ND', 'SD', 'WV', 'WY',
       'AL', 'PR', 'GU', 'VI', 'U.S.'], dtype=object)

In [271]:
df['Free Treatment'] = 0
df['Early RX Refills'] = 0
df['SEP'] = 0
df['Waiver 1135'] = 0
df['Paid Sick Leave'] = 0

In [272]:
# No cost for treatment
df['Free Treatment'] = np.where((df['Last Update'] >= '2020-03-12') & (df.State == 'D.C.'), 1, df['Free Treatment'])
df['Free Treatment'] = np.where((df['Last Update'] >= '2020-03-06') & (df.State == 'MA'), 1, df['Free Treatment'])
df['Free Treatment'] = np.where((df['Last Update'] >= '2020-03-13') & (df.State == 'NM'), 1, df['Free Treatment'])

In [273]:
df['Free Treatment'].value_counts()

0    1513
1      47
Name: Free Treatment, dtype: int64

In [274]:
# Early Prescription Refills
df['Early RX Refills'] = np.where((df['Last Update'] >= '2020-03-05') & (df.State == 'WA'), 1, df['Early RX Refills'])
df['Early RX Refills'] = np.where((df['Last Update'] >= '2020-03-18') & (df.State == 'CA'), 1, df['Early RX Refills'])
df['Early RX Refills'] = np.where((df['Last Update'] >= '2020-03-03') & (df.State == 'AK'), 1, df['Early RX Refills'])
df['Early RX Refills'] = np.where((df['Last Update'] >= '2020-03-09') & (df.State == 'CO'), 1, df['Early RX Refills'])
df['Early RX Refills'] = np.where((df['Last Update'] >= '2020-03-17') & (df.State == 'LA'), 1, df['Early RX Refills'])
df['Early RX Refills'] = np.where((df['Last Update'] >= '2020-03-20') & (df.State == 'GA'), 1, df['Early RX Refills'])
df['Early RX Refills'] = np.where((df['Last Update'] >= '2020-03-10') & (df.State == 'FL'), 1, df['Early RX Refills'])
df['Early RX Refills'] = np.where((df['Last Update'] >= '2020-03-09') & (df.State == 'KY'), 1, df['Early RX Refills'])
df['Early RX Refills'] = np.where((df['Last Update'] >= '2020-03-13') & (df.State == 'WV'), 1, df['Early RX Refills'])
df['Early RX Refills'] = np.where((df['Last Update'] >= '2020-03-12') & (df.State == 'ME'), 1, df['Early RX Refills'])
df['Early RX Refills'] = np.where((df['Last Update'] >= '2020-03-10') & (df.State == 'NH'), 1, df['Early RX Refills'])
df['Early RX Refills'] = np.where((df['Last Update'] >= '2020-03-26') & (df.State == 'MA'), 1, df['Early RX Refills'])
df['Early RX Refills'] = np.where((df['Last Update'] >= '2020-03-13') & (df.State == 'RI'), 1, df['Early RX Refills'])
df['Early RX Refills'] = np.where((df['Last Update'] >= '2020-03-09') & (df.State == 'DE'), 1, df['Early RX Refills'])
df['Early RX Refills'] = np.where((df['Last Update'] >= '2020-03-06') & (df.State == 'MD'), 1, df['Early RX Refills'])
df['Early RX Refills'] = np.where((df['Last Update'] >= '2020-03-12') & (df.State == 'DC'), 1, df['Early RX Refills'])

In [275]:
df['Early RX Refills'].value_counts()

0    1393
1     167
Name: Early RX Refills, dtype: int64

In [276]:
# SEP
df['SEP'] = np.where((df['Last Update'] >= '2020-02-18') & (df.State == 'CA'), 1, df['SEP'])
df['SEP'] = np.where((df['Last Update'] >= '2020-03-19') & (df.State == 'CO'), 1, df['SEP'])
df['SEP'] = np.where((df['Last Update'] >= '2020-03-17') & (df.State == 'CT'), 1, df['SEP'])
df['SEP'] = np.where((df['Last Update'] >= '2020-02-25') & (df.State == 'D.C.'), 1, df['SEP'])
df['SEP'] = np.where((df['Last Update'] >= '2020-03-13') & (df.State == 'MD'), 1, df['SEP'])
df['SEP'] = np.where((df['Last Update'] >= '2020-03-11') & (df.State == 'MA'), 1, df['SEP'])
df['SEP'] = np.where((df['Last Update'] >= '2020-03-17') & (df.State == 'NV'), 1, df['SEP'])
df['SEP'] = np.where((df['Last Update'] >= '2020-03-16') & (df.State == 'NY'), 1, df['SEP'])
df['SEP'] = np.where((df['Last Update'] >= '2020-03-23') & (df.State == 'RI'), 1, df['SEP'])
df['SEP'] = np.where((df['Last Update'] >= '2020-03-20') & (df.State == 'VT'), 1, df['SEP'])
df['SEP'] = np.where((df['Last Update'] >= '2020-03-10') & (df.State == 'WA'), 1, df['SEP'])
df['SEP'] = np.where((df['Last Update'] >= '2020-03-20') & (df.State == 'MN'), 1, df['SEP'])

In [277]:
df['SEP'].value_counts()

0    1323
1     237
Name: SEP, dtype: int64

In [278]:
# Section 1135 waiver
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-23') & (df.State == 'AL'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-23') & (df.State == 'AZ'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-23') & (df.State == 'CA'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-26') & (df.State == 'CO'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-27') & (df.State == 'CT'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-27') & (df.State == 'DE'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-16') & (df.State == 'FL'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-26') & (df.State == 'HI'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-26') & (df.State == 'ID'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-23') & (df.State == 'IL'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-25') & (df.State == 'IN'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-25') & (df.State == 'IA'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-25') & (df.State == 'KS'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-25') & (df.State == 'KY'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-23') & (df.State == 'LA'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-26') & (df.State == 'MD'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-26') & (df.State == 'MA'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-27') & (df.State == 'MN'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-23') & (df.State == 'MS'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-25') & (df.State == 'MO'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-23') & (df.State == 'NH'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-23') & (df.State == 'NJ'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-23') & (df.State == 'NM'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-26') & (df.State == 'NY'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-23') & (df.State == 'NC'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-24') & (df.State == 'ND'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-24') & (df.State == 'OK'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-25') & (df.State == 'OR'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-27') & (df.State == 'PA'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-25') & (df.State == 'RI'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-24') & (df.State == 'SD'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-23') & (df.State == 'VA'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-15') & (df.State == 'WA'), 1, df['Waiver 1135'])
df['Waiver 1135'] = np.where((df['Last Update'] >= '2020-03-27') & (df.State == 'WY'), 1, df['Waiver 1135'])

In [279]:
df['Waiver 1135'].value_counts()

0    1547
1      13
Name: Waiver 1135, dtype: int64

In [280]:
# Paid sick leave
df['Paid Sick Leave'] = np.where((df['Last Update'] >= '2018-01-01') & (df.State == 'WA'), 1, df['Paid Sick Leave'])
df['Paid Sick Leave'] = np.where((df['Last Update'] >= '2016-01-01') & (df.State == 'OR'), 1, df['Paid Sick Leave'])
df['Paid Sick Leave'] = np.where((df['Last Update'] >= '2018-07-01') & (df.State == 'CA'), 1, df['Paid Sick Leave'])
df['Paid Sick Leave'] = np.where((df['Last Update'] >= '2017-07-01') & (df.State == 'AZ'), 1, df['Paid Sick Leave'])
df['Paid Sick Leave'] = np.where((df['Last Update'] >= '2020-03-10') & (df.State == 'CO'), 1, df['Paid Sick Leave'])
df['Paid Sick Leave'] = np.where((df['Last Update'] >= '2020-03-17') & (df.State == 'NY'), 1, df['Paid Sick Leave'])
df['Paid Sick Leave'] = np.where((df['Last Update'] >= '2018-01-01') & (df.State == 'VT'), 1, df['Paid Sick Leave'])
df['Paid Sick Leave'] = np.where((df['Last Update'] >= '2015-07-01') & (df.State == 'MA'), 1, df['Paid Sick Leave'])
df['Paid Sick Leave'] = np.where((df['Last Update'] >= '2018-07-01') & (df.State == 'RI'), 1, df['Paid Sick Leave'])
df['Paid Sick Leave'] = np.where((df['Last Update'] >= '2012-01-01') & (df.State == 'CT'), 1, df['Paid Sick Leave'])
df['Paid Sick Leave'] = np.where((df['Last Update'] >= '2018-10-01') & (df.State == 'NJ'), 1, df['Paid Sick Leave'])
df['Paid Sick Leave'] = np.where((df['Last Update'] >= '2018-02-01') & (df.State == 'MD'), 1, df['Paid Sick Leave'])
df['Paid Sick Leave'] = np.where((df['Last Update'] >= '2014-01-01') & (df.State == 'D.C.'), 1, df['Paid Sick Leave'])

#df['Paid Sick Leave'] = np.where((df['Last Update'] >= '2019-03-01') & (df.State == 'MI'), 1, df['Paid Sick Leave'])
#df['Paid Sick Leave'] = np.where((df['Last Update'] >= '2020-01-01') & (df.State == 'NV'), 1, df['Paid Sick Leave'])

In [281]:
df['Paid Sick Leave'].value_counts()

0    873
1    687
Name: Paid Sick Leave, dtype: int64

In [282]:
df

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude,Province,State,Free Treatment,Early RX Refills,SEP,Waiver 1135,Paid Sick Leave
36,Unassigned Location,US,2020-02-26 20:03:06,42.0,0.0,0.0,NaN,NaN,Unassigned Location,Unassigned Location,0,0,0,0,0
69,"Chicago, IL",US,2020-02-09 19:03:03,2.0,0.0,2.0,NaN,NaN,Chicago,IL,0,0,0,0,0
70,"San Benito, CA",US,2020-02-03 03:53:02,2.0,0.0,0.0,NaN,NaN,San Benito,CA,0,0,0,0,1
71,"San Diego County, CA",US,2020-02-21 05:43:02,2.0,0.0,1.0,NaN,NaN,San Diego County,CA,0,0,1,0,1
72,"Santa Clara, CA",US,2020-02-21 05:23:04,2.0,0.0,1.0,NaN,NaN,Santa Clara,CA,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,"Yolo County, CA",US,2020-03-06 20:13:14,1.0,0.0,0.0,38.7646,-121.9018,Yolo County,CA,0,0,1,0,1
251,"Lackland, TX",US,2020-02-24 23:33:02,0.0,0.0,0.0,29.3829,-98.6134,Lackland,TX,0,0,0,0,0
252,"Montgomery County, TX",US,2020-03-07 19:53:02,0.0,0.0,0.0,30.3213,-95.4778,Montgomery County,TX,0,0,0,0,0
253,"Omaha, NE",US,2020-02-24 23:33:02,0.0,0.0,0.0,41.2545,-95.9758,Omaha,NE,0,0,0,0,0


In [283]:
df.shape

(1560, 15)

In [284]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1560 entries, 36 to 254
Data columns (total 15 columns):
Province/State      1560 non-null object
Country/Region      1560 non-null object
Last Update         1560 non-null datetime64[ns]
Confirmed           1560 non-null float64
Deaths              1530 non-null float64
Recovered           1530 non-null float64
Latitude            1186 non-null float64
Longitude           1186 non-null float64
Province            1560 non-null object
State               1529 non-null object
Free Treatment      1560 non-null int64
Early RX Refills    1560 non-null int64
SEP                 1560 non-null int64
Waiver 1135         1560 non-null int64
Paid Sick Leave     1560 non-null int64
dtypes: datetime64[ns](1), float64(5), int64(5), object(4)
memory usage: 195.0+ KB
